# Código para leitura de dados no excel e no AFIM para criação de um Banco de Dados

### Importa Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.remote.errorhandler import NoAlertPresentException
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.keys import Keys

from navegacao import para

import navegacao
import pandas as pd

from tkinter import Tk, filedialog, messagebox, simpledialog

import os
import sqlite3


### Carrega as funções 

In [2]:
def adicionar_informacoes_ao_banco_de_dados(dataframe, nome_banco_de_dados, nome_tabela):
    colunas = dataframe.columns

    # Exclui linhas duplicadas no dataframe, usando o metodo 'duplicated' e negando os verdadeiros com '~'.
    dataframe = dataframe[~dataframe.duplicated()]

    # Dicionário de mapeamento para renomear as colunas tirando espaços para '_'
    novo_nome_colunas = dict(zip(colunas, [c.replace(' ', '_') for c in colunas]))

    # Use o método rename
    dataframe.rename(columns=novo_nome_colunas, inplace=True)

    # if messagebox.askyesno("Confirmação", "Você deseja adicionar informações ao banco de dados?"):
    # Conecta-se ao banco de dados
    conn = sqlite3.connect(nome_banco_de_dados)
    
    # Adiciona as informações do DataFrame à tabela no banco de dados
    dataframe.to_sql(nome_tabela, conn, if_exists='append', index=False)

    verificar_e_tratar_duplicatas(nome_tabela, conn)
    
    # Fecha a conexão
    conn.close()

    return "Informações adicionadas ao banco de dados!"
        # messagebox.showinfo("Sucesso", "Informações adicionadas ao banco de dados!")

def get_year():
    janela = Tk()
    year = simpledialog.askinteger("Informe o ano", "Digite o ano:")
    # if year is not None:  # Verifica se o usuário forneceu um valor válido
    janela.destroy()
    return year

def verificar_e_tratar_duplicatas(table_name, conn):
    # Conectar ao banco de dados
    cursor = conn.cursor()

    # Obter informações sobre as colunas da tabela
    cursor.execute(f"PRAGMA table_info({table_name})")
    column_info = cursor.fetchall()

    # Construir uma lista de nomes de colunas
    column_names = [info[1] for info in column_info]

    # Construir uma consulta SQL dinâmica para verificar duplicatas
    sql_check_duplicates = f'''
        SELECT *, COUNT(*) as qtd_duplicadas
        FROM {table_name}
        GROUP BY {', '.join(column_names)}
        HAVING COUNT(*) > 1
    '''

    # Executar a consulta para verificar duplicatas
    cursor.execute(sql_check_duplicates)
    duplicates = cursor.fetchall()

    if duplicates:
        print("Linhas duplicadas encontradas:")
        for duplicate in duplicates:
            print(f"Quantidade de duplicatas: {duplicate[-1]}")

        # Construir uma consulta SQL dinâmica para tratar duplicatas
        # (remover todas, exceto a primeira ocorrência)
        sql_remove_duplicates = f'''
            DELETE FROM {table_name}
            WHERE ROWID NOT IN (
                SELECT MIN(ROWID)
                FROM {table_name}
                GROUP BY {', '.join(column_names)}
                HAVING COUNT(*) > 1
            )
        '''

        # Executar a consulta para tratar duplicatas
        cursor.execute(sql_remove_duplicates)
        print("Linhas duplicadas tratadas.")

        # Confirmar e fechar a conexão
        conn.commit()
    else: print("Nenhuma linha duplicada encontrada.")

def incluir_coluna_no_banco_de_dados(nome_banco_de_dados, nome_tabela, nome_coluna, valores_coluna=None):
    # if messagebox.askyesno("Confirmação", "Você deseja incluir uma nova coluna no banco de dados?"):
        # Conecta-se ao banco de dados
        conn = sqlite3.connect(nome_banco_de_dados)
        
        # Cria uma consulta SQL para adicionar uma nova coluna
        consulta = f'ALTER TABLE {nome_tabela} ADD COLUMN "{nome_coluna}"'
        
        # Executa a consulta para adicionar a nova coluna
        conn.execute(consulta)
        
        if valores_coluna:
            # Atualiza a tabela com os valores da nova coluna
            for valor in valores_coluna:
                consulta = f'UPDATE {nome_tabela} SET "{nome_coluna}" = ?'
                conn.execute(consulta, (valor,))
        
        # Commit para salvar as alterações
        conn.commit()
        
        # Fecha a conexão
        conn.close()
        # messagebox.showinfo("Sucesso", "Nova coluna incluída no banco de dados!")

def carregar_arquivos():
    janela = Tk()
    file = filedialog.askopenfilename(title="Selecione o arquivo para carregar")
    janela.destroy()

    return file

def salvar_arquivos():
    janela = Tk()
    file = filedialog.asksaveasfilename(title="Selecione o destino para salvar o arquivo")
    janela.destroy()

    return file

def alerta_existe(self, termo, tempo=5):
    '''
        SE EXISTIR ALERTA, RETORNA VERDADEIRO
        
        SE NÃO EXISTIR, RETORNA FALSO
    '''
    from selenium.webdriver.remote.errorhandler import NoAlertPresentException
    from navegacao import para
    para(tempo)
    try: 
        a = self.navegador.switch_to.alert
        if termo in a.text: a.accept()
        return True
    except NoAlertPresentException: return False

def ler_extrato(ne):
    downloads = os.path.expanduser("~"+os.sep+"Downloads")
    caminho = os.path.join(downloads, 'RELEXTRATOEMPENHO.xls')
    # "C:\Users\luan.pinto\Downloads\RELEXTRATOEMPENHO.xls"

    df = pd.read_excel(caminho)[['Evento', 'Documento', 'Data', 'Valor']][:-2]
    df['EMPENHO'] = ne

    os.remove(caminho)
    return df

def clonar_banco(db_origem, db_destino, origem_table):
    # Conectar ao banco de dados de origem
    conn_origem = sqlite3.connect(db_origem)
    dataframe = pd.read_sql_query(f"SELECT * FROM {origem_table}", conn_origem)
    conn_origem.close()

    adicionar_informacoes_ao_banco_de_dados(dataframe, db_destino, origem_table)

def excluir_tabela(tabela, conn):
    # Crie um objeto cursor
    cursor = conn.cursor()
    # Execute o comando SQL para excluir a tabela
    cursor.execute(f'DROP TABLE IF EXISTS {tabela}')
    # Confirme a operação
    conn.commit()
    # Feche a conexão com o banco de dados
    conn.close()

def atualiza_dados(nome_banco_de_dados, sua_tabela, atualizar_coluna, coluna_criterio, novo_valor, critério):
    # Conectar ao banco de dados
    conn = sqlite3.connect(nome_banco_de_dados)
    cursor = conn.cursor()

    # Atualizar dados na tabela
    cursor.execute(f'UPDATE {sua_tabela} SET {atualizar_coluna} = ? WHERE {coluna_criterio} = ?', (novo_valor, critério))

    # Commit para salvar as alterações
    conn.commit()

    # Fechar a conexão
    conn.close()



## Criando uma função para buscar as OBs dos contratos

### Tratamento do Dataframe
- OBS.: Tirar a dúvida com o DPI, se é uma empresa por contrato
1. Buscar as OBs nos anos posteriores 


Cálculo do Contrato:

1. **Soma** dos Valores dos Empenhos
2. **Menos** (-) o valor das Anulações
3. **Menos** (-) o valor das OBs

**Saldo = soma(Empenhos) - soma(Anulações) - soma(OBs)**

In [10]:
db = r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\DATABASE\OBRAS.db'
# ano = get_year()
ano_ = 2021
conn = sqlite3.connect('OBRAS.db')
df_contratos = pd.read_sql_query("SELECT * FROM Contratos", conn)
df_relsalemp = pd.read_sql_query("SELECT * FROM RELSALEMP", conn)
df_relsalemp_completo = pd.read_sql_query("SELECT * FROM Relsalemp_completo", sqlite3.connect(db))



In [ ]:

incluir_no_banco_de_dados = []
empresas = list(set(df_relsalemp['CREDOR']))
for e,empresa in enumerate(empresas):
    print(f'= = = = = = {e+1}/{len(empresas)}: {empresa} = = = = = =')
    cnpj, nome = empresa.split(' - ')

    n = navegacao.Navegacao(modo_escondido=False)

    lista_obs = []
    for ano in range(ano_, 2024):
        n.entra_AFIM(ano)
        lista_obs.append(n.LISOB(cnpj))
        n.navegador.execute_script("window.open('', '_blank');")
        n.trocar_abas('blank')
    n.navegador.quit()

    empenho = df_relsalemp[df_relsalemp['CREDOR']==empresa][:1]['EMPENHO'].values[0]
    df_aux = df_relsalemp_completo[df_relsalemp_completo['EMPENHO']==empenho][:1]

    for ob in lista_obs:
        for o in ob:
            aux = []
            aux.append('Pagamento por NL')
            aux.extend(o.split(' ')[3:6])
            aux.extend([df_aux['EMPENHO'].values[0],
                        df_aux['UNIDADE_ORÇAMENTARIA'].values[0],
                        df_aux['PROGRAMA_DE_TRABALHO'].values[0],
                        df_aux['FONTE_DE_RECURSO'].values[0],
                        df_aux['NATUREZA_DESPESA'].values[0],
                        df_aux['CONTA_CONTÁBIL'].values[0],
                        df_aux['CONTRATO_N°'].values[0]])
            incluir_no_banco_de_dados.append(aux)

incluir_no_banco_de_dados


In [11]:
df_relsalemp_completo

,Evento,Documento,Data,Valor,EMPENHO,UNIDADE_ORÇAMENTARIA,PROGRAMA_DE_TRABALHO,FONTE_DE_RECURSO,NATUREZA_DESPESA,CONTA_CONTÁBIL,CONTRATO_N°
0,Empenho,2021NE00542,09/06/2021,271269.96,2021NE00542,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...,018/2021
1,Em Liquidação,2021NL02390,25/08/2021,164458.28,2021NE00542,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...,018/2021
2,Liquidação,2021NL02465,02/09/2021,164458.28,2021NE00542,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...,018/2021
3,Pagamento por NL,2021NL02465,02/09/2021,2466.87,2021NE00542,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...,018/2021
4,Pagamento por NL,2021NL02465,02/09/2021,822.29,2021NE00542,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...,018/2021
...,...,...,...,...,...,...,...,...,...,...,...
298,Pagamento por NL,2022OB01344,"7.657,62",03/11/2022,2021NE01471,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,06100354 - RECURSOS DO PROMINF / MANAUS / FINISA,"44905193 - REFORMAS, BENFEITORIA OU MELHORIA",123210112 - REFOR. BENFEIT. OU MELHORIS,039/2021
299,Pagamento por NL,2022OB01345,"42.643,35",03/11/2022,2021NE01471,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,06100354 - RECURSOS DO PROMINF / MANAUS / FINISA,"44905193 - REFORMAS, BENFEITORIA OU MELHORIA",123210112 - REFOR. BENFEIT. OU MELHORIS,039/2021
300,Pagamento por NL,2022OB01346,"1.109,95",03/11/2022,2021NE01471,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,06100354 - RECURSOS DO PROMINF / MANAUS / FINISA,"44905193 - REFORMAS, BENFEITORIA OU MELHORIA",123210112 - REFOR. BENFEIT. OU MELHORIS,039/2021
301,Pagamento por NL,2022OB01347,"554,98",03/11/2022,2021NE01471,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,06100354 - RECURSOS DO PROMINF / MANAUS / FINISA,"44905193 - REFORMAS, BENFEITORIA OU MELHORIA",123210112 - REFOR. BENFEIT. OU MELHORIS,039/2021


In [13]:

# Exibindo DataFrame antes da atualização
print("DataFrame antes da atualização:")
# print(df_relsalemp_completo)

# Condição para selecionar as linhas onde '/' está presente em 'Valor'
condicao = df_relsalemp_completo['Valor'].str.contains('/')

# Trocando os valores entre 'Data' e 'Valor' nas linhas que atendem à condição
df_relsalemp_completo.loc[condicao, ['Data', 'Valor']] = df_relsalemp_completo.loc[condicao, ['Valor', 'Data']].values

# Exibindo DataFrame depois da atualização
print("\nDataFrame depois da atualização:")
# print(df_relsalemp_completo)


DataFrame antes da atualização:

DataFrame depois da atualização:


In [14]:
adicionar_informacoes_ao_banco_de_dados(df_relsalemp_completo, db, 'Relsalemp_completo')

Nenhuma linha duplicada encontrada.


'Informações adicionadas ao banco de dados!'

Codigo para testar se no LISPD é possivel pesquisar por NEs

## Une informações

### Carrega o banco de dados **OBRAS.db**

In [ ]:
conn = sqlite3.connect(r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\DATABASE\OBRAS.db')
df_relsalemp = pd.read_sql_query("SELECT * FROM RELSALEMP", conn)
df_Relsalemp_completo = pd.read_sql_query("SELECT * FROM Relsalemp_completo", conn)
conn.close()


In [ ]:

df_excel = pd.read_excel(r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\ARQUIVOS\obras_2021.xlsx')

colunas = ['OBJETO', 'CONTRATO N°', 'EMPRESA', 'SITUAÇÃO DA OBRA', 'VALOR DO CONTRATO (R$)']

df_excel = df_excel[colunas]
df_excel.rename(columns={'VALOR DO CONTRATO (R$)': 'VALOR DO CONTRATO'}, inplace=True)

adicionar_informacoes_ao_banco_de_dados(df_excel, r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\DATABASE\OBRAS.db', 'Contratos')


## Pegando informações no LISPD

In [ ]:
db = r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\DATABASE\OBRAS.db'
conn = sqlite3.connect(db)
df_contratos = pd.read_sql_query("SELECT * FROM Contratos", conn)
df_relsalemp = pd.read_sql_query("SELECT * FROM RELSALEMP", conn)
df_contratos_excel = pd.read_excel(r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS\contratos de obras - 2021.xlsx")


### Atualizando os credores com CNPJ no RELSALEMP

In [ ]:
nave = navegacao.Navegacao()

nave.entra_AFIM(2021)


for ne in list(set(df_relsalemp['EMPENHO'])):
    v = [i.text for i in nave.LISNE(ne)]
    atualiza_dados(db,'RELSALEMP', 'CREDOR', 'EMPENHO', v[2], ne)